In [1]:
import requests
import pandas as pd
from io import StringIO
import plotly.graph_objects as go
from tqdm import trange
import time
from random import randint

base_url = "https://www.space-track.org"
login_path = "/ajaxauth/login"

credentials = {
    "identity": "laidlaw.eliot@gmail.com",
    "password": "sbudnicisfalling",
}

transporter5_sat_num_range = [52732, 52782]
sbudnic_sat_num = 52774


def get_space_data_path(sat_num: int) -> str:
    return f"/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/EPOCH asc/format/csv/emptyresult/show"


def get_random_gray_color() -> str:
    def random_component() -> int:
        return randint(0, 180)

    return f"rgb({random_component()}, {random_component()}, {random_component()})"

In [2]:
session = requests.Session()

resp = session.post(f"{base_url}{login_path}", data=credentials)
if resp.status_code != 200:
    raise Exception("Login failed.")

In [36]:
dfs = {}
retry_count = 3

for sat_num in trange(*transporter5_sat_num_range):
    for i in range(retry_count):
        resp = session.get(f"{base_url}{get_space_data_path(sat_num)}")
        if resp.status_code != 200:
            print("retrying...")
            continue

        dfs[sat_num] = pd.read_csv(StringIO(resp.text), sep=",")

        time.sleep(2)
        break

100%|██████████| 50/50 [02:01<00:00,  2.42s/it]


In [63]:
fig = go.Figure()
for sat_num in dfs:
    df = dfs[sat_num]
    is_sbudnic = sat_num == sbudnic_sat_num
    fig.add_trace(
        go.Scatter(
            x=df["EPOCH"],
            y=df["PERIAPSIS"],
            name=df["OBJECT_NAME"][50],
            line=go.scatter.Line(
                width=4 if is_sbudnic else 1.5,
                color="gold" if is_sbudnic else get_random_gray_color(),
            ),
        )
    )

fig.update_layout(
    font_family="Arial",
    font_color="black",
    title="Transporter 5 Satellite Perigees vs. Time",
    xaxis_title="Date",
    yaxis_title="Perigee (km)",
)

fig.show()
fig.write_html("index.html")
